In [1]:
import Functions.StringComp as SC
import Models.ImportModel as IM
import pandas as pd
import json
import re
import numpy as np



In [2]:
# Import the unique researcher spreadsheet
#base = pd.read_excel('base.xlsx')
base = pd.read_excel('duplicates_reduced.xlsx')

# Import the personnummer
pnum = pd.read_excel('pnums.xlsx')

scrape5 = pd.read_excel('scrape_results_5.0.xlsm')


In [3]:
def get_pnum(app_id):

    subset = pnum[(pnum['app_id'] == app_id)]

    x = subset.personnummer
    isnull = x.isnull().values.any()

    if isnull == True:
        return []

    # correct for multiple pnums
    try:
        if re.search(r"[\[\]]", x.iloc[0]) != None:
            x = re.sub(r"[\[\]\']","", x.iloc[0]).split(", ")
            return list(set(x))
    except:
        return []

    # if just one pnum return in a list
    else:
        return [x.iloc[0]]


In [4]:
all_pnums = []

for t in base.itertuples():
    local_pnums = []
    app_id_list = re.sub(r"[\[\]\']", "", t.app_ids).split(", ")
    for app_id in app_id_list:
        local_pnums = local_pnums + get_pnum(app_id)

    all_pnums.append(list(set(local_pnums)))

print(all_pnums)


[['19830529-3386'], ['19770510-0241'], [], [], [], ['19830330-0704'], ['19810708-0809'], [], [], [], ['19810801-0441'], [], [], ['19840405-2741'], [], [], ['19740908-8585'], ['19750420-4988'], [], [], ['19831022-1711'], [], [], [], [], ['19790430-3646'], ['19791116-3330'], ['19780503-2443'], [], [], [], [], ['19810529-4824'], [], ['19810810-7866'], [], [], [], [], [], ['19051017-2012', '19670903-7516'], [], [], [], ['19780128-4618'], [], ['19720912-5546'], [], [], [], [], ['19780714-0087'], [], [], [], ['19740427-8595'], [], [], [], ['19770504-0173'], [], ['19730904-8480'], ['19841029-3537'], ['19791015-6657'], [], [], [], ['19780519-7832'], [], [], ['19810416-4994'], [], [], [], [], [], [], [], [], ['19820510-5581'], [], [], [], [], [], [], [], [], [], [], [], [], ['19790203-3997'], [], [], [], ['19750327-9056'], [], [], ['19810504-4062'], [], [], [], [], [], ['19820608-7416'], [], [], [], ['19801119-3938'], [], [], [], ['19750107-5969'], [], [], [], [], [], [], ['19791012-1982'], [],

In [5]:
# remove list from single-pnum lists
clean = []
for x in all_pnums:
    if len(x) == 1:
        clean.append(x[0])
    elif len(x) == 0:
        clean.append(None)
    else:
        clean.append(x)

print("LENGTH WAS", len(all_pnums))

all_pnums_clean = clean.copy()
print("LENGTH IS", len(all_pnums_clean))

print(all_pnums_clean)

LENGTH WAS 13546
LENGTH IS 13546
['19830529-3386', '19770510-0241', None, None, None, '19830330-0704', '19810708-0809', None, None, None, '19810801-0441', None, None, '19840405-2741', None, None, '19740908-8585', '19750420-4988', None, None, '19831022-1711', None, None, None, None, '19790430-3646', '19791116-3330', '19780503-2443', None, None, None, None, '19810529-4824', None, '19810810-7866', None, None, None, None, None, ['19051017-2012', '19670903-7516'], None, None, None, '19780128-4618', None, '19720912-5546', None, None, None, None, '19780714-0087', None, None, None, '19740427-8595', None, None, None, '19770504-0173', None, '19730904-8480', '19841029-3537', '19791015-6657', None, None, None, '19780519-7832', None, None, '19810416-4994', None, None, None, None, None, None, None, None, '19820510-5581', None, None, None, None, None, None, None, None, None, None, None, None, '19790203-3997', None, None, None, '19750327-9056', None, None, '19810504-4062', None, None, None, None, None

In [6]:
print(base.head())

df2 = base.assign(personnummer_cv=all_pnums_clean)
print(df2.head())


              name             surname  \
0            Amaia  Calderón-Larrañaga   
1            Maria            Swanberg   
2  Christine-Maria              Horejs   
3            Antti             Latvala   
4          Ljubica             Perisic   

                                             app_ids  duplicates  
0                                     ['2016-00981']           1  
1  ['2016-00992', '2013-03310', '2012-02563', '20...           6  
2                                     ['2016-01005']           1  
3                                     ['2016-01008']           1  
4                                     ['2016-01016']           1  
              name             surname  \
0            Amaia  Calderón-Larrañaga   
1            Maria            Swanberg   
2  Christine-Maria              Horejs   
3            Antti             Latvala   
4          Ljubica             Perisic   

                                             app_ids  duplicates  \
0                       

In [7]:
# export
writer = pd.ExcelWriter('test.xlsx', engine='xlsxwriter')
df2.to_excel(writer, sheet_name="Researchers", index=False)
writer.save()

In [8]:
def extract_cell(app_id, col):
    subset = scrape5[(scrape5['Dnr'] == app_id)]
    x = getattr(subset, col)
    isnull = x.isnull().values.any()
    if isnull:
        return []
    else:
        val = str(x.iloc[0])

        if ".0" == val[-2:]:
            val = val[:-2]

        # if there is a comma serparated number list, split it and return a list
        if re.search(r"^[\d,\.\s]+$", val) != None:
            if ", " in val:
                return val.split(", ")
            else:
                return [val]
        else:
            return [val]



print(extract_cell("2016-00981", 'Year'))
print(extract_cell("2016-00992", "Year"))

print(extract_cell("2016-01158", 'Year'))
print(extract_cell("2016-01172", "Year_unsure"))

print(extract_cell("2016-00981", 'year_of_birth'))
print(extract_cell("2016-00992", "year_of_birth"))
print(extract_cell("2016-01158", 'year_of_birth'))
print(extract_cell("2016-01172", "year_of_birth"))

print(extract_cell("2016-00981", 'Institution'))
print(extract_cell("2016-00992", "Institution"))
print(extract_cell("2016-01158", 'Institution'))
print(extract_cell("2016-01172", "Institution"))

[]
['2007']
[]
['2015', '2006']
[]
['1977']
[]
[]
[]
['Karolinska Institutet']
[]
['Karolinska Institutet']


In [9]:
def clean_list(list):

    # remove list from single-pnum lists
    clean = []
    for x in list:
        if len(x) == 1:
            # convert to integer if it looks like one
            if re.match(r"^\d+$", str(x[0])) != None:
                clean.append(int(x[0]))
            else:
                clean.append(x[0])

        elif len(x) == 0:
            clean.append(None)

        else:
            clean.append(", ".join(x))

    print("LENGTH WAS", len(list))
    print("LENGTH IS", len(clean))

    return(clean)


print(clean_list([[], ['2007'], [], ['2011'], ['2012']]))



LENGTH WAS 5
LENGTH IS 5
[None, 2007, None, 2011, 2012]


In [10]:
# Now we aggregate and append desired columns

def aggregate_attribute(a):

    a_list = []

    for t in df2.itertuples():
        local_list = []
        app_id_list = re.sub(r"[\[\]\']", "", t.app_ids).split(", ")
        for app_id in app_id_list:
            local_list = local_list + extract_cell(app_id, a)

        #print(list(set(local_list)))

        a_list.append(list(set(local_list)))

    return clean_list(a_list)


In [11]:
years = aggregate_attribute('Year')
years_unsure = aggregate_attribute("Year_unsure")
birth_years = aggregate_attribute("year_of_birth")
institutions = aggregate_attribute("Institution")

LENGTH WAS 13546
LENGTH IS 13546
LENGTH WAS 13546
LENGTH IS 13546
LENGTH WAS 13546
LENGTH IS 13546
LENGTH WAS 13546
LENGTH IS 13546


In [12]:
df3 = df2.assign(year=years,
                 years_unsure=years_unsure,
                 birth_year=birth_years,
                 institution=institutions)

In [13]:
df3.head()

# export
writer = pd.ExcelWriter('RA_Sheet.xlsx', engine='xlsxwriter')
df3.to_excel(writer, sheet_name="Researchers", index=False)
writer.save()



